In [4]:
import json 
f = open('report-val.json')
overall_stats = f.readline()

In [20]:
while True:
    g = json.loads(f.readline())
    candidate = g['code']['info']['candidates'][0]
    sequence = g['code']['code_sequence']
    ex_code = g['example']['code']
    correct = g['stats']['correct']
    

6 6
11 6
6 6
9 6
12 6
9 6
19 0
13 6
13 0
8 6
7 6
23 0
14 0
14 0
23 0
7 6
8 6
21 0
11 0
18 0
8 6
11 6
16 6
9 6
13 0
10 6
13 6
7 6
12 6
22 6
17 5
7 6
11 6
8 6
9 6
13 6
17 0
18 6
9 6
21 0
18 6
17 6
8 6
6 6
14 0
7 6
7 6
8 6
6 6
30 0
22 0
8 6
18 0
9 6
7 6
21 6
20 0
25 0
9 6
16 6
14 6
16 0
7 6
16 0
13 0
9 5
19 6
6 6
7 6
8 6
9 6
7 6
9 5
22 0
8 6
10 6
0 0
11 6
7 6
17 6
7 6
8 6
9 6
10 6
18 0
20 0
10 6
7 6
13 6
14 0
8 6
9 6
14 0
7 6
14 6
6 6
7 6
15 6
17 6
18 6
17 0
6 6
6 6
22 0
14 6
15 0
6 6
10 6
12 0
15 0
9 6
7 6
15 0
7 6
12 6
7 6
18 0
18 6
8 6
14 6
13 0
6 6
6 6
16 6
6 6
7 6
14 6
14 0
18 0
15 0
18 6
20 0
15 5
7 6
17 0
13 0
13 6
8 6
9 6
20 0
13 0
13 6
19 0
7 6
10 6
8 6
8 6
12 6
6 6
7 6
12 0
14 0
20 0
10 6
11 6
17 6
16 0
15 0
16 0
22 0
23 0
8 6
14 6
8 6
13 0
6 6
7 6
7 6
13 6
7 6
19 6
14 0
18 0
8 6
14 0
14 6
13 0
21 0
8 6
7 6
19 0
10 6
6 6
7 6
9 6
14 6
14 0
8 6
11 6
7 6
6 6
18 0
23 0
8 6
17 6
7 6
12 6
18 0
7 6
7 6
11 0
10 6
17 0
14 6
9 6
6 6
15 5
6 6
12 0
14 0
12 0
6 6
6 6
12 0
8 6
9 6
6 6
16 6
9 

ValueError: Expecting value: line 1 column 1 (char 0)

In [1]:
import numpy as np

data = np.loadtxt('data.csv', delimiter=',')
labels = np.loadtxt('labels.csv', delimiter=',')

# Explore different ways to canonicalize the I/O
 1. Data is coming in as (2500, 78) -> first reshape to (2500, 6, 13)
 2. Sort rows based on a column value
 3. reshape back to (2500, 78)

In [7]:
from sklearn.linear_model import LogisticRegression

io_data = data[:, 18:]
reshaped_data = io_data.reshape((2500, 6, 13))

val1, val2, train1, train2 = [], [], [], []

for i in range(13):  
    fresh_data = np.zeros((2500, 78))
    for t in range(2500): # t for trial
        rows = reshaped_data[t]
        sorted_rows = rows[rows[:,i].argsort()]
        fresh_data[t] = sorted_rows.reshape(78)
    
    #print(data[:, :18].shape)
    #print(fresh_data.shape)
    reformatted_data = np.concatenate((data[:, :18], fresh_data), axis=1)
    for c in range(96):
        reformatted_data /= np.max(np.abs(reformatted_data[:,c])) + 1e-4
        reformatted_data[:,c] -= np.mean(reformatted_data[:,c])
        

    log_reg = LogisticRegression()
    log_reg.fit(reformatted_data[:2000], labels[:2000])
    train1.append(log_reg.score(reformatted_data[:2000], labels[:2000]))
    val1.append(log_reg.score(reformatted_data[2000:], labels[2000:]))
    
    log_reg.fit(reformatted_data[500:], labels[500:])
    train2.append(log_reg.score(reformatted_data[500:], labels[500:]))
    val2.append(log_reg.score(reformatted_data[:500], labels[:500]))

print(val1)
print(train1)
print(np.argsort(val1))
print(np.argsort(train1))
print(np.argsort(val2))
print(np.argsort(train2))

[0.82799999999999996, 0.81999999999999995, 0.80800000000000005, 0.81000000000000005, 0.79600000000000004, 0.79200000000000004, 0.81200000000000006, 0.79600000000000004, 0.79000000000000004, 0.79800000000000004, 0.80600000000000005, 0.79000000000000004, 0.77800000000000002]
[0.8165, 0.8135, 0.8115, 0.81299999999999994, 0.8175, 0.81899999999999995, 0.82199999999999995, 0.80400000000000005, 0.80900000000000005, 0.80800000000000005, 0.82199999999999995, 0.8165, 0.81999999999999995]
[12  8 11  5  4  7  9 10  2  3  6  1  0]
[ 7  9  8  2  3  1  0 11  4  5 12  6 10]
[ 8 10  7 12  1  6  2  9  5  0 11  3  4]
[ 8  2  7  9 10  4 11  3  1  6  0  5 12]


In [4]:
# logistic regression isn't super impressive ~82% accuracy. 
# let's try basic neural network instead

from keras.models import Sequential
from keras.layers import Dense, Activation, Flatten

model = Sequential()

model.add(Dense(100, activation='relu', input_shape=(96,)))
model.add(Dense(100, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

model.compile(loss='binary_crossentropy', 
             optimizer='adam', 
             metrics=['accuracy'])

io_data = data[:, 18:]
reshaped_data = io_data.reshape((2500, 6, 13))

# val1, val2, train1, train2 = [], [], [], []

for i in range(13):  
    fresh_data = np.zeros((2500, 78))
    for t in range(2500): # t for trial
        rows = reshaped_data[t]
        sorted_rows = rows[rows[:,i].argsort()]
        fresh_data[t] = sorted_rows.reshape(78)
    
    #print(data[:, :18].shape)
    #print(fresh_data.shape)
    reformatted_data = np.concatenate((data[:, :18], fresh_data), axis=1)
    for c in range(96):
        reformatted_data /= np.max(np.abs(reformatted_data[:,c])) + 1e-4
        reformatted_data[:,c] -= np.mean(reformatted_data[:,c])
       

    model.fit(reformatted_data[:2000], labels[:2000], 
             batch_size=16, nb_epoch=5, verbose=1)
    print(model.evaluate(reformatted_data[2000:], labels[2000:], verbose=1))

Epoch 1/5
2000/2000 [==============================] - 1s - loss: 1.6288 - acc: 0.6805      
Epoch 2/5
2000/2000 [==============================] - 0s - loss: 0.6714 - acc: 0.7635     

In [29]:
lin_model.get_params()

{'C': 1.0,
 'class_weight': None,
 'dual': False,
 'fit_intercept': True,
 'intercept_scaling': 1,
 'max_iter': 100,
 'multi_class': 'ovr',
 'n_jobs': 1,
 'penalty': 'l2',
 'random_state': None,
 'solver': 'liblinear',
 'tol': 0.0001,
 'verbose': 0,
 'warm_start': False}